In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt
!rm data/ILSVRC2012_img_val.tar

--2022-05-04 00:56:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  11.8MB/s    in 9m 5s   

2022-05-04 01:05:38 (11.8 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-04 01:06:09--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.ResNet152(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

242909184/242900224 [==============================] - 4s 0us/step


In [5]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.resnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.704 Top 5:0.908
2%, Top 1:0.71 Top 5:0.905
3%, Top 1:0.716 Top 5:0.9053333333333333
4%, Top 1:0.706 Top 5:0.8995
5%, Top 1:0.7156 Top 5:0.9028
6%, Top 1:0.7146666666666667 Top 5:0.9016666666666666
7%, Top 1:0.7128571428571429 Top 5:0.904
8%, Top 1:0.7165 Top 5:0.903
9%, Top 1:0.7175555555555555 Top 5:0.9015555555555556
10%, Top 1:0.716 Top 5:0.901
11%, Top 1:0.7158181818181818 Top 5:0.8998181818181819
12%, Top 1:0.7156666666666667 Top 5:0.9011666666666667
13%, Top 1:0.7175384615384616 Top 5:0.9007692307692308
14%, Top 1:0.7188571428571429 Top 5:0.902
15%, Top 1:0.7201333333333333 Top 5:0.9028
16%, Top 1:0.71975 Top 5:0.902375
17%, Top 1:0.7188235294117648 Top 5:0.9016470588235294
18%, Top 1:0.7166666666666667 Top 5:0.9011111111111111
19%, Top 1:0.7171578947368421 Top 5:0.9012631578947369
20%, Top 1:0.7169 Top 5:0.9012
21%, Top 1:0.716952380952381 Top 5:0.9017142857142857
22%, Top 1:0.7165454545454546 Top 5:0.901
23%, Top 1:0.7174782608695652 Top 5:0.9013043478260869
24%, To